## Example of out-of-probe model
This data set contains data already partitioned, with the training data of 9 probes and the test data
from an independent probe.

In [1]:
# Import Packages
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import voltammetry
import matplotlib as mpl
from sklearn.metrics import r2_score
from recordclass import recordclass
# load data 
LABS_out_of_probe_data = h5py.File('pool_uncorrelated_100k_97Hz_25_DA_5HT_pH_WS_1500_TSS_125_CF45.h5','r')
training_vgrams = LABS_out_of_probe_data['training/vgrams']
training_labels = LABS_out_of_probe_data['training/labels']
testing_vgrams = LABS_out_of_probe_data['testing/vgrams']
testing_labels = LABS_out_of_probe_data['testing/labels']

### Function Defintions

In [2]:
# define function to build the models [ for brevity, only use alpha 1.0]
def build_ensemble(vgrams, labels):
    # initialize variables
    model_array = []
    training = recordclass('training','labels, vgrams')
    # define bin limits
    lim_DA = [900,1800,2700] # max in nanomoles
    lim_5HT = [900,1800,2700] # max in nanomoles
    bins = [(0,0)] # starting concentration limits
    bin_count = 0 # number of bins starting from 0 - 9
    # loop through serotonin bins
    for i, c_5HT in enumerate(lim_5HT):
        # define the maximum and minimum concentrations for iteration
        if i == 0:
            c_5HT_min = -0.0001
            c_5HT_max = c_5HT
        else:
            c_5HT_min = lim_5HT[i-1]
            c_5HT_max = c_5HT
        # loop through dopamine concentrations
        for j, c_DA in enumerate(lim_DA):
            if j == 0:
                c_DA_min = -0.0001
                c_DA_max = c_DA
            else:
                c_DA_min = lim_DA[j-1]
                c_DA_max = c_DA
            print(''.join(('bin: ',str(int(bin_count)),
                           ' | DA [',str(int(c_DA_min)),',',str(int(c_DA_max)),
                           '] | 5-HT [',str(int(c_5HT_min)),',',str(int(c_5HT_max)),']')))
            # select data within concentration bin
            idx = np.squeeze(np.where((labels[:,0] > c_DA_min) &
                                     (labels[:,0] <= c_DA_max) &
                                     (labels[:,1] > c_5HT_min) &
                                     (labels[:,1] <= c_5HT_max)))
            training.labels = np.squeeze(labels[idx,:])
            training.vgrams = np.squeeze(vgrams[idx,:])
            training.labels = training.labels[::25,:]
            training.vgrams = training.vgrams[::25,:]
            # Build models for model array
            # bestAlpha = voltammetry.best_alpha(training,parallel=40) #~ in HPC application
            bestAlpha = 1.0
            cvFit = voltammetry.train_analyte(training, alpha=bestAlpha,parallel=8)
            model_array.append(cvFit)
            bins.append((c_DA,c_5HT))
            bin_count = bin_count +1
    
    return model_array, bins

In [3]:
# define function to search match models to the testing data
def ensemble_predictions(vgrams,labels,model_array):
    # initialize variables
    predictions = np.zeros((np.shape(labels)[0],np.shape(labels)[1]))
    testing = recordclass('testing', 'labels, vgrams')
    testing_bin = np.empty((9))
    b_count = 0
    # loop through serotonin bins
    for i, c_5HT in enumerate(lim_5HT):
        # define the maximum and minimum concentrations for iteration
        if i == 0:
            c_5HT_min = -0.0001
            c_5HT_max = c_5HT
        else:
            c_5HT_min = lim_5HT[i-1]
            c_5HT_max = c_5HT
        # loop through dopamine concentrations
        for j, c_DA in enumerate(lim_DA):
            if j == 0:
                c_DA_min = -0.0001
                c_DA_max = c_DA
            else:
                c_DA_min = lim_DA[j-1]
                c_DA_max = c_DA
            print(''.join(('Predicting bin: ',str(int(bin_count)),
                           ' | DA [',str(int(c_DA_min)),',',str(int(c_DA_max)),
                           '] | 5-HT [',str(int(c_5HT_min)),',',str(int(c_5HT_max)),']')))
            # select data within concentration bin
            idx = np.squeeze(np.where((labels[:,0] > c_DA_min) &
                                     (labels[:,0] <= c_DA_max) &
                                     (labels[:,1] > c_5HT_min) &
                                     (labels[:,1] <= c_5HT_max)))
            testing.labels = np.squeeze(labels[idx,:])
            testing.vgrams = np.squeeze(vgrams[idx,:])
            # iterate through model array to find best model
            pred_list = []
            for k, cvFit in enumerate(model_array):
                # generate predictions
                pred = np.squeeze(voltammetry.test_analyte(testing,cvFit))
                pred_list.append(pred)
                # calculate error eij with respect to bin
                mean_DA = (c_DA_min + c_DA_max)/2
                mean_5HT = (c_5HT_min + c_5HT_max)/2
                err_cv[i] = np.sum((pred[:,0] - mean_DA)**2,axis=0) + np.sum((pred[:,1]-mean_5HT)**2,axis=0)
            # identify model with minimum error
            min_err_idx = np.argmin(err_cv)
            print(min_err_idx)
            cvFit = cvFitList[min_err_idx]
            predictions[idx,:] = pred_list[min_err_idx]

            mdl_select_bin[b_count] = min_err_idx
            b_count = b_count + 1
    return predictions, mdl_select_bin

In [ ]:
# Plot regression of predictions with known values.
# Return figure, 3 axes for dopamine, serotonin and pH.
def plot_regr(true_val,predictions):
    # initialize plotting variables
    fig, ax = plt.subplots(1,3,dpi=100,figsize=(6,3))
    x = true_val
    y = predictions
    analytes=['dopamine','serotonin','pH']
    analyte_colors = ['#1f77b4','#b49e1f','#3ebd30']
    xlim = [(-.500,3.200),(-.500,3.200),(6.6,8.0)]
    ylim = xlim
    textLoc = [(0.5,2.800),(0.5000,2.800),(6.9,7.9)]
    linlim = [(0,2.700),(0,2.700),(6.8,7.8)]
    for chemIx in range(len(analytes)):
        labColor = analyte_colors[chemIx]
        steps = np.unique(x[:,chemIx]) # unique concentrations
        mean_val = np.empty(len(steps))
        stdv_val = np.empty(len(steps))
        # calculate mean and stdev at each concentration
        for i,val in enumerate(steps):
            idx = np.where(x[:,chemIx]==val)
            mean_val[i] = np.mean(y[idx,chemIx])
            stdv_val[i] = statistics.stdev(y[idx,chemIx][0,:,0])
        # calculate line of best fit
        [m,b] = np.polyfit(x[:,chemIx],y[:,chemIx],1)
        # plot line of best fit
        ax[chemIx].plot(linlim[chemIx],m*linlim[chemIx]+b,'k',
                        label=''.join(('y=','{:.3}'.format(m[0]),'*x+','{:.3f}'.format(b[0]))))
        r2 = r2_score(x[:,chemIx],y[:,chemIx])
        # plot mean and standard deviation by concentration
        ax[chemIx].errorbar(steps,mean_val,yerr=stdv_val,color=labColor,fmt='.',label='Mean $\pm$ StDev')
        # format plot
        ax[chemIx].text(textLoc[chemIx][0],textLoc[chemIx][1],''.join(['R$^2$=''{0:.3f}'.format(r2)]))
        ax[chemIx].set_xlabel(''.join(['known ',analytes[chemIx],r' $({\rm\mu M})$']))
        ax[chemIx].set_ylabel(''.join(['predicted ',analytes[chemIx],r' $({\rm\mu M})$']))
        ax[chemIx].set_xlim(xlim[chemIx])
        ax[chemIx].set_ylim(ylim[chemIx])
        ax[chemIx].legend(loc='lower right')
        plt.tight_layout()
    # format axis for pH units
    ax[2].set_ylabel(''.join(['prediction ',analytes[chemIx],' (pH)']))
    ax[2].set_xlabel(''.join(['known ',analytes[chemIx],' (pH)']))
    return fig, ax

In [ ]:
# plot model selection
def mdl_select(mdl_select_bin):
    # initialize grid
    fig,ax = plt.subplots(dpi=100)
    gl = [0.9,0.9,0.9] # grid color
    #ax.imshow(0.9*np.ones((3,3)),cmap='gray')# draw grid
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    for i in range(len(testing_bin)):
        data_bin = i
        mdl_bin = mdl_select_bin[i]
        # fill in the selected model bin for the test data
        ax.fill_between([data_bin,data_bin+1],[md_bin,md_bin+1])
        ax.plot([i,i],[0,9],color=gl)
        ax.plot([0,9],[i,i],color=gl)
    ax.plot([0,9],[9,9],color=gl)
    ax.plot([9,9],[0,9])
    ax.set_xlabel('test data bin')
    ax.set_ylabel('selected model bin')
    return fig,ax
        
          


### Ensemble model training

To train a model, the training data from nine of the ten probes are partitioned according to concentration [900 nM DA X 900 nM 5-HT] and EN-penalized regression is used to generate models within each bin.

For the purposes of saving computational resources and time in this demo, only $\alpha$ of 1.0 will be used. The training data will also be downsampled to by a factor of 25. If performing on a full data set, it is recommended to use a high performance computing cluster with multiple cores.

In [4]:
# define function to plot results# run  ensemble model on data

In [5]:
# Define funciton to plot model selection

In [ ]:
# run  ensemble model on data
[model_array,bins] = build_ensemble(training_vgrams,training_labels)
# generate predictions
[predictions,mdl_select_bin] = ensemble_predictions(testing_vgrams,testing_labels,model_array)

bin: 0 | DA [0,900] | 5-HT [0,900]


### Visualize results

In [ ]:
# Plot results as regression of known and predicted values
[reg_fig,reg_ax] = plot_regr(testing_labes,predictions)

In [ ]:
# Map the relationship between model and test data bins
for i,b in enumerate(bins):
    print('bin ',i,':',b)
[fig_map,ax_map] = mdl_select(mdl_select_bin)


In [ ]:
## Credits
import sys, platform, time
print('This data demo was created using:')
print('Python Version:',sys.version)
print('Operating System:',platform.system(),'Version',platform.release())
print('GLMnet for python: https://web.stanford.edu/~hastie/glmnet_python/')
print('Numpy: https://numpy.org/')
print('h5py: http://www.h5py.org/')
print('pyplot: https://matplotlib.org/index.html')
print('sklearn: https://scikit-learn.org/stable/')
print('recordclass: https://pypi.org/project/recordclass/')
print('Last updated:',time.strftime('%d-%b-%Y %H:%M:%S',time.localtime()))